In [1]:
import pandas as pd
import regex as re
import numpy as np
import ast
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import math
import unicodedata
import re
from urllib.request import urlopen
import json


#101 brands with Good On You 4/5
PRODUCTS_PATH = './data/goodonyou_five_products_reduced.csv'
df = pd.read_csv(PRODUCTS_PATH) #final_table

df = df.drop(['IMAGE', 'IN_STOCK'], axis=1).drop_duplicates()
df['BRAND_URL'] = df['BRAND_URL'].apply(lambda x: x.replace("\n",""))

In [10]:
def extract_text(body_html):
    try:
        bs_list = BeautifulSoup(body_html,'html.parser').find_all(text=True)
        desc = "".join(bs_list).replace('\n',' ')
        return unicodedata.normalize("NFKD", desc)
    except:
        if body_html=="nan" or type(body_html)==float:
            return ""
        else:
            return body_html
        
#### TODO: This will no longer be needed if we re-run the scraper
df['DESCRIPTION'] = df['BODY_HTML'].apply(extract_text)
df = df.drop(['BODY_HTML'], axis=1).drop_duplicates()

In [11]:
df.head()

,BRAND_URL,PRODUCT_CATEGORY,PRODUCT,PRICE,PRODUCT_URL,DESCRIPTION
0,https://www.tasitravels.com,Dress,Adrift Dress Flower Fields,219.0,\nhttps://www.tasitravels.com/products/adrift-...,Shipping end March Our classic button through ...
1,https://www.tasitravels.com,Dress,Sunrise Slip Dress Moss,189.0,\nhttps://www.tasitravels.com/products/sunrise...,Our Sunrise Slip Dress is ready for whenever ...
2,https://www.tasitravels.com,Pant,Journey Pant Paisley Blush,169.0,\nhttps://www.tasitravels.com/products/journey...,The Journey pull-on pant creates versatile and...
4,https://www.tasitravels.com,Top,Wilder Wrap Blouse Native Floral,139.0,\nhttps://www.tasitravels.com/products/wilder-...,The Wilder Wrap Blouse is your carefree compan...
5,https://www.tasitravels.com,Dress,Sunrise Slip Dress Paper Daisy,189.0,\nhttps://www.tasitravels.com/products/sunrise...,Our Sunrise Slip Dress is ready for whenever a...


In [12]:
df.shape

(22943, 6)

In [13]:
len(set(df.PRODUCT))

16471

##### Alternate Extract Materials Algo

Before:
Create a .csv file with first column as material name and later columns as regexes for material

Algo:

Import Material_Regex csv

Create key and matching regex dictionary

Create list of only regexes

For each row of DF-

    Convert to lower
    For each regex-
    
        Check if "num% regex" is in row with re.finall (include re.MULTILINE)
        From first tuple extract number for percent and add to dictionary
        
    If dictionary empty add the description to "Others" list for review to add to regex sheet

In [14]:
GranRegs = pd.read_csv('./data/materials_proxy.csv',header=None) 

In [15]:
num_cols = 0
with open('./data/materials_proxy.csv', 'r') as f:
    num_cols = len(f.readline().split())

a = GranRegs. iloc[:, 0] 
b = np.genfromtxt('./data/materials_proxy.csv',dtype=str,delimiter=',',usecols=range(1,num_cols)) 

d = dict(zip(a,b.tolist()))    #if you want a numpy array, just remove .tolist()

d

{'Acetate, Triacetate fabric': [' Triacetate fabric"',
  '"acetate',
  ' triacetate fabric"',
  'acetate triacetate',
  'acetate'],
 'Acrylic fabric': ['acrylic', '', '', '', ''],
 'Alpaca Fabric': ['alpaca', '', '', '', ''],
 'Aramid fabric': ['aramid', '', '', '', ''],
 'Carbon fiber fabric': ['carbon', '', '', '', ''],
 'Viscose/Rayon (generic), regenerated cellulose from wood pulp': [' regenerated cellulose from wood pulp"',
  '"viscose/rayon \\(generic\\)',
  ' regenerated cellulose from wood pulp"',
  'regenerated viscose',
  'regenerated rayon'],
 'Cotton fabric': ['cotton', '', '', '', ''],
 'Cotton fiber, organic': [' organic"',
  '"cotton fiber',
  ' organic"',
  'organic ?-?cotton',
  'cotton organic'],
 'Cotton fiber, recycled': [' recycled"',
  '"cotton fiber',
  ' recycled"',
  'recycled ?-?cotton',
  ''],
 'Elastane/Spandex  fabric': ['elastane\\/spandex ',
  'elastane',
  'spandex',
  'elastane ?-?spandex',
  ''],
 'Flax fiber fabric': ['flax', '', '', '', ''],
 'Glass 

In [16]:
#reverse key and value
#match regex to material
regex_to_material = {}
for key, val in d.items():
    for regex in val:
        if len(regex)!=0:
            regex_to_material[regex] = key

In [17]:
regex_to_material

{' Triacetate fabric"': 'Acetate, Triacetate fabric',
 '"acetate': 'Acetate, Triacetate fabric',
 ' triacetate fabric"': 'Acetate, Triacetate fabric',
 'acetate triacetate': 'Acetate, Triacetate fabric',
 'acetate': 'Acetate, Triacetate fabric',
 'acrylic': 'Poly(methyl methacrylate)(PMMA) (Acrylic) plastic',
 'alpaca': 'Alpaca Fabric',
 'aramid': 'Aramid fabric',
 'carbon': 'Carbon fiber fabric',
 ' regenerated cellulose from wood pulp"': 'Viscose/Rayon (generic), regenerated cellulose from wood pulp',
 '"viscose/rayon \\(generic\\)': 'Viscose/Rayon (generic), regenerated cellulose from wood pulp',
 'regenerated viscose': 'Viscose/Rayon (generic), regenerated cellulose from wood pulp',
 'regenerated rayon': 'Viscose/Rayon (generic), regenerated cellulose from wood pulp',
 'cotton': 'Cotton fabric',
 ' organic"': 'Cotton fiber, organic',
 '"cotton fiber': 'Cotton fiber, recycled',
 'organic ?-?cotton': 'Cotton fiber, organic',
 'cotton organic': 'Cotton fiber, organic',
 ' recycled"': 

In [18]:
#iterate all rows of df to get material from description
materials_perc = [] #list of dictionaries
others = [] #list of clothes description that did not match
for index, row in df.iterrows():
    row_materials = {}
    for regex,materialkey in regex_to_material.items():
        perc_regex = '\d+%[a-z ]*'+regex
        regex = re.compile(perc_regex)
        results = re.findall(regex,row['DESCRIPTION'].lower())
        if len(results)!=0:
            first = results[0]
            #split of %
            perc, rest = first.split('%')
            row_materials[materialkey] = perc+'%'
    if not row_materials:
        others.append(row)
    materials_perc.append(row_materials)

In [19]:
len(others)

14900

In [20]:
len(df['DESCRIPTION'])

22943

In [21]:
materials_perc

[{'Modal fabric': '100%', 'Wood': '100%'},
 {},
 {'Modal fabric': '100%'},
 {'Modal fabric': '100%'},
 {'Modal fabric': '100%'},
 {'Modal fabric': '100%'},
 {'Modal fabric': '100%'},
 {'Modal fabric': '100%'},
 {},
 {'Modal fabric': '100%'},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {'Cotton fabric': '100%'},
 {},
 {'Cotton fabric': '100%', 'Cotton fiber, organic': '100%'},
 {'Polylactic Acid (PLA) fabric': '100%'},
 {'Cotton fabric': '100%'},
 {},
 {'Cotton fabric': '100%'},
 {'Polylactic Acid (PLA) fabric': '100%'},
 {'Cotton fabric': '100%'},
 {'Polylactic Acid (PLA) fabric': '100%'},
 {'Cotton fabric': '100%'},
 {},
 {},
 {},
 {'Elastane/Spandex  fabric': '10%',
  'Hemp fiber fabric': '20%',
  'Modal fabric': '100%',
  'Polypropylene (PP), recycled, for plastic': '90%'},
 {},
 {'Hemp fiber fabric': '20%'},
 {'Cotton fabric': '100%', 'Cotton fiber, organic': '100%'},
 {'Cotton fabric': '100%', 'Cotton fiber, organic': '100%'},
 {'Elastane/

In [22]:
df['material_perc'] = materials_perc

In [23]:
list(zip(df['DESCRIPTION'], df['material_perc']))

[('Shipping end March Our classic button through sundress is an easy favourite for all seasons. Throw on over swimwear, layer and style as a duster, or even add boots and a jacket. With a waist-cinching back tie, relaxed bodice and floaty skirt for the most flattering of fits, the Adrift Dress is a reliable go-to. Handcrafted in Australia from 100%  Certified Lenzing ModalR.  Why Lenzing ModalR? Lenzing ModalR is a beautifully soft yet strong sustainable fabric, created from 100% certified renewable wood pulp that is derived mainly from Beech Trees. Not only a dream to wear, this special fabric also possesses effective moisture wicking properties to keep you comfortable for all seasons.  Our Model Lala is size M/L and is 174cm/5’7” tall.   Handcrafted in Australia Biodegradable Coconut shell buttons  Adjustable waist tie Relaxed sleeve Bra friendly  Gentle cold machine wash, more info here    ',
  {'Modal fabric': '100%', 'Wood': '100%'}),
 (' Our Sunrise Slip Dress is ready for whenev

In [24]:
df.to_csv(r'./data/MichelleMaterialClassification.csv', index = False)

In [25]:
df['MATERIAL'] = df['DESCRIPTION'].apply(extract_materials)

NameError: name 'extract_materials' is not defined

In [ ]:
list(zip(df['DESCRIPTION'], df['MATERIAL']))

Only 24.8% matched (5690/22943 products)

1) not in a "#% material" format
2) doesn't include materials information in description
3) unknown materials (doesn't match with Higgs like TENCELL)

In [ ]:
sum([ len(e) > 0 for e in df['MATERIAL'] ]) / len(df['MATERIAL'])